In [ ]:
!pip install -q transformers==4.51.3 datasets==2.14.4 torch==2.6.0 ipywidgets==7.7.1 rouge_score


In [ ]:
!pip install gradio

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
import os
base_path = '/content/drive/MyDrive/BBC News Summary'
articles_dir = os.path.join(base_path, 'News Articles')
model_save_path = '/content/drive/MyDrive/News_Generator_Final'
os.makedirs(model_save_path, exist_ok=True)


In [ ]:
import pandas as pd
import glob
from datetime import datetime

def cargar_articulos(categoria):
    articulos = []
    for archivo in glob.glob(os.path.join(articles_dir, categoria, '*.txt')):
        with open(archivo, 'r', encoding='latin-1') as f:
            contenido = f.read().split('\n')
            titulo = contenido[0].strip()
            cuerpo = ' '.join([linea.strip() for linea in contenido[1:] if linea.strip()])
            articulos.append({
                'category': categoria.upper(),
                'title': titulo,
                'content': cuerpo[:1800]
            })
    return articulos

categorias = ['business', 'entertainment', 'politics', 'sport', 'tech']
df = pd.DataFrame([art for cat in categorias for art in cargar_articulos(cat)])


In [ ]:
from datetime import datetime as dt
hoy_str = dt.now().strftime('%Y-%m-%d')

df['text'] = df.apply(
    lambda x: f"""\
[{x['category']}]
FECHA: {hoy_str}
TITULAR: {x['title']}
CONTENIDO: {x['content']}
---""",
    axis=1
)


In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def funcion_tokenizacion(ejemplos):
    return tokenizer(
        ejemplos['text'],
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

from datasets import Dataset
dataset = Dataset.from_pandas(df[['text']])
dataset = dataset.map(funcion_tokenizacion, batched=True, batch_size=32, remove_columns=['text'])

dataset = dataset.train_test_split(test_size=0.1)


In [ ]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling

modelo = GPT2LMHeadModel.from_pretrained('distilgpt2')
modelo.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

argumentos_entrenamiento = TrainingArguments(
    output_dir='./resultados',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    eval_steps=500,
    save_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    report_to="none"
)

entrenador = Trainer(
    model=modelo,
    args=argumentos_entrenamiento,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)



In [ ]:
print(" Iniciando entrenamiento...")
entrenador.train()


In [ ]:
model_save_path = "/content/drive/MyDrive/News_Generator_Final"

entrenador.model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)



In [ ]:
import gradio as gr

def generar_desde_gradio(categoria, longitud, temperatura):
    prompt = f"[{categoria}]\nFECHA: {dt.now().strftime('%Y-%m-%d')}\nTITULAR:"
    generado = generador(
        prompt,
        max_length=longitud,
        temperature=temperatura,
        top_p=0.92,
        repetition_penalty=1.35,
        num_return_sequences=1,
        no_repeat_ngram_size=2
    )
    contenido = generado[0]['generated_text'].split("CONTENIDO:", 1)[-1].strip()
    return contenido

categorias_opciones = [c.upper() for c in categorias]

gr.Interface(
    fn=generar_desde_gradio,
    inputs=[
        gr.Dropdown(choices=categorias_opciones, label="Categoría"),
        gr.Slider(200, 800, step=50, value=400, label="Longitud"),
        gr.Slider(0.3, 1.2, step=0.05, value=0.72, label="Creatividad")
    ],
    outputs=gr.Textbox(lines=20, label="Noticia Generada"),
    title="📰 Generador de Noticias BBC",
    description="Modelo de texto entrenado con artículos BBC | DistilGPT2"
).launch(share=True)